In [1]:
import pandas as pd
import numpy as np
from apyori import apriori

In [2]:
article = pd.read_csv("articles.csv")
customer = pd.read_csv("customers.csv")
sample_submission = pd.read_csv("sample_submission.csv")
transaction_train = pd.read_csv("transactions_train.csv")

In [3]:
transaction_train.head()

t_dat                                        customer_id  article_id  \
0  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   663713001   
1  2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...   541518023   
2  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   505221004   
3  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687003   
4  2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699...   685687004   

      price  sales_channel_id  
0  0.050831                 2  
1  0.030492                 2  
2  0.015237                 2  
3  0.016932                 2  
4  0.016932                 2

In [4]:
transaction_train.describe()

article_id         price  sales_channel_id
count  3.178832e+07  3.178832e+07      3.178832e+07
mean   6.962272e+08  2.782927e-02      1.704028e+00
std    1.334480e+08  1.918113e-02      4.564786e-01
min    1.087750e+08  1.694915e-05      1.000000e+00
25%    6.328030e+08  1.581356e-02      1.000000e+00
50%    7.145820e+08  2.540678e-02      2.000000e+00
75%    7.865240e+08  3.388136e-02      2.000000e+00
max    9.562170e+08  5.915254e-01      2.000000e+00

In [5]:
transaction_train.isnull().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [6]:
pd.options.mode.chained_assignment = None

ref: https://medium.com/@jihargifari/how-to-perform-market-basket-analysis-in-python-bd00b745b106

doc:http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#example-3-working-with-sparse-representations

In [7]:
transaction_train = transaction_train[['t_dat','customer_id','article_id']]

In [8]:
print(min(transaction_train['t_dat']),max(transaction_train['t_dat']))

2018-09-20 2020-09-22


In [16]:
df = transaction_train[transaction_train['t_dat']>='2020-07-01']

In [17]:
df.shape

(3386963, 3)

In [19]:
# assume items bought by the same customer on the same day is on the same receipt (basket)
df['dummy_invoice']=df.groupby(['t_dat','customer_id']).ngroup()

In [20]:
df.head(20)

t_dat                                        customer_id  \
28401361  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401362  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401363  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401364  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401365  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401366  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401367  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401368  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401369  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401370  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401371  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401372  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401373  2020-07-01  0008968c0d451dbc5a9968da03196fe20051965edde741...   
28401374  2020-07-01  000ce5da167c6c8dfaea6dfc4b59a5ea3217630ec36cfc...   
28401375  2020-07-01  000ce5da167c6c8dfaea6dfc4b59a5ea3217630ec36cfc...   
28401376  2020-07-01  000ce5da167c6c8dfaea6dfc4b59a5ea3217630ec36cfc...   
28401377  2020-07-01  000fb6e772c5d0023892065e659963da90b1866035558e...   
28401378  2020-07-01  000fb6e772c5d0023892065e659963da90b1866035558e...   
28401379  2020-07-01  000fb6e772c5d0023892065e659963da90b1866035558e...   
28401380  2020-07-01  00130faf36f2571cf7e08451b317545004a8a85327661f...   

          article_id  dummy_invoice  
28401361   825577003              0  
28401362   849487005              0  
28401363   772659001              0  
28401364   804732004              0  
28401365   804732004              0  
28401366   804732003              0  
28401367   887949004              0  
28401368   824194004              0  
28401369   876461001              0  
28401370   869796001              0  
28401371   562498011              0  
28401372   788576001              0  
28401373   850985001              1  
28401374   559616013              2  
28401375   559616013              2  
28401376   566140001              2  
28401377   858172002              3  
28401378   524529004              3  
28401379   907696002              3  
28401380   617787001              4

In [22]:
df['no_unique_article'] = df.groupby('dummy_invoice')['article_id'].transform('nunique')
df.head(15)

t_dat                                        customer_id  \
28401361  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401362  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401363  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401364  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401365  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401366  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401367  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401368  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401369  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401370  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401371  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401372  2020-07-01  0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...   
28401373  2020-07-01  0008968c0d451dbc5a9968da03196fe20051965edde741...   
28401374  2020-07-01  000ce5da167c6c8dfaea6dfc4b59a5ea3217630ec36cfc...   
28401375  2020-07-01  000ce5da167c6c8dfaea6dfc4b59a5ea3217630ec36cfc...   

          article_id  dummy_invoice  no_unique_article  
28401361   825577003              0                 11  
28401362   849487005              0                 11  
28401363   772659001              0                 11  
28401364   804732004              0                 11  
28401365   804732004              0                 11  
28401366   804732003              0                 11  
28401367   887949004              0                 11  
28401368   824194004              0                 11  
28401369   876461001              0                 11  
28401370   869796001              0                 11  
28401371   562498011              0                 11  
28401372   788576001              0                 11  
28401373   850985001              1                  1  
28401374   559616013              2                  2  
28401375   559616013              2                  2

In [32]:
df.tail()

t_dat                                        customer_id  \
31788319  2020-09-22  fff2282977442e327b45d8c89afde25617d00124d0f999...   
31788320  2020-09-22  fff2282977442e327b45d8c89afde25617d00124d0f999...   
31788321  2020-09-22  fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...   
31788322  2020-09-22  fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...   
31788323  2020-09-22  fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...   

          article_id  dummy_invoice  no_unique_article  
31788319   929511001        1040900                  2  
31788320   891322004        1040900                  2  
31788321   918325001        1040901                  1  
31788322   833459002        1040902                  1  
31788323   898573003        1040903                  1

In [23]:
basket_plus = df[df['no_unique_article']>1]
print(df.shape,basket_plus.shape)

(3386963, 5) (3002103, 5)


In [24]:
# cast to str due to an error from frequent itemset mining later
basket_plus.article_id = [str(i) for i in basket_plus.article_id]

In [25]:
transactions = basket_plus.groupby('dummy_invoice')['article_id'].apply(list)
transactions

dummy_invoice
0          [825577003, 849487005, 772659001, 804732004, 8...
2                          [559616013, 559616013, 566140001]
3                          [858172002, 524529004, 907696002]
4               [617787001, 564313022, 764147003, 854247002]
5                          [822311010, 843772002, 701472001]
                                 ...                        
1040896                               [895730002, 740519002]
1040897    [919273002, 881448002, 919273001, 919273004, 8...
1040898                               [918522001, 918485001]
1040899    [896169005, 791587021, 791587021, 910949002, 7...
1040900                               [929511001, 891322004]
Name: article_id, Length: 696233, dtype: object

In [28]:
# try sparse
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
oht_ary = te.fit(transactions).transform(transactions, sparse=True)
basket_encode = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
basket_encode

108775015  108775044  110065001  110065002  110065011  111565001  \
0               0          0          0          0          0          0   
1               0          0          0          0          0          0   
2               0          0          0          0          0          0   
3               0          0          0          0          0          0   
4               0          0          0          0          0          0   
...           ...        ...        ...        ...        ...        ...   
696228          0          0          0          0          0          0   
696229          0          0          0          0          0          0   
696230          0          0          0          0          0          0   
696231          0          0          0          0          0          0   
696232          0          0          0          0          0          0   

        111586001  111593001  111609001  120129001  ...  948152002  949198001  \
0               0          0          0          0  ...          0          0   
1               0          0          0          0  ...          0          0   
2               0          0          0          0  ...          0          0   
3               0          0          0          0  ...          0          0   
4               0          0          0          0  ...          0          0   
...           ...        ...        ...        ...  ...        ...        ...   
696228          0          0          0          0  ...          0          0   
696229          0          0          0          0  ...          0          0   
696230          0          0          0          0  ...          0          0   
696231          0          0          0          0  ...          0          0   
696232          0          0          0          0  ...          0          0   

        949551001  949551002  949594001  952267001  952938001  953450001  \
0               0          0          0          0          0          0   
1               0          0          0          0          0          0   
2               0          0          0          0          0          0   
3               0          0          0          0          0          0   
4               0          0          0          0          0          0   
...           ...        ...        ...        ...        ...        ...   
696228          0          0          0          0          0          0   
696229          0          0          0          0          0          0   
696230          0          0          0          0          0          0   
696231          0          0          0          0          0          0   
696232          0          0          0          0          0          0   

        953763001  956217002  
0               0          0  
1               0          0  
2               0          0  
3               0          0  
4               0          0  
...           ...        ...  
696228          0          0  
696229          0          0  
696230          0          0  
696231          0          0  
696232          0          0  

[696233 rows x 39835 columns]

In [65]:
# for ref only, this is not sparse matrix
# from mlxtend.preprocessing import TransactionEncoder

# te = TransactionEncoder()
# te_ary = te.fit(transactions).transform(transactions)
# basket_encode = pd.DataFrame(te_ary, columns=te.columns_)
# basket_encode

In [29]:
from mlxtend.frequent_patterns import apriori

In [37]:
frequent_itemsets = apriori(basket_encode, min_support=0.002,
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

frequent_itemsets

support     itemsets  length
0   0.005843  (751471001)       1
1   0.005699  (706016001)       1
2   0.005227  (372860002)       1
3   0.004491  (610776002)       1
4   0.004361  (730683050)       1
..       ...          ...     ...
75  0.002071  (915526002)       1
76  0.002038  (865929003)       1
77  0.002031  (907696002)       1
78  0.002018  (711053003)       1
79  0.002004  (599580061)       1

[80 rows x 3 columns]

In [39]:
frequent_itemsets[frequent_itemsets['length']>1]

Empty DataFrame
Columns: [support, itemsets, length]
Index: []

In [34]:
from mlxtend.frequent_patterns import association_rules

In [36]:
association_rules(frequent_itemsets, metric ='lift',
                 min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []

In [ ]:
# calculate quantity of item on the same receipt
# basket = df.groupby(['dummy_invoice','article_id']).size()
# basket = pd.DataFrame(basket).reset_index()
# basket.rename(columns={basket.columns[2]: 'quantity'}, inplace = True)
# basket

In [ ]:
# have not started - experiment with https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this/notebook

In [18]:
vc = df.article_id.value_counts()

In [19]:
vc

706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
             ...  
254457030        1
581293001        1
868812001        1
867569001        1
551124007        1
Name: article_id, Length: 104547, dtype: int64

In [26]:
df.loc[df.article_id==534357002,'customer_id']

6612        23b1f7858432cffa3130687990c0a7fb09f73badfebf3c...
14982       4f44b9a8d6c12db5a703d22b0fa08d6d433bbeda020436...
18312       5fe3e8928770a90d287d620fade678325b6e2c04e04fd7...
18347       5ffd00f17e6effccd88b49c56c265c4d0150e4a4514e87...
26781       8c70d5ad7e345ed27a817d1baa393c72190d1e7c928ad2...
                                  ...                        
29577667    364cf7ff690d4f2190a8ed4354ee1ffd7b055d0e20f7ef...
29588874    74fc5dd0036a0d0db26379f13e9e8e760569c46d16abf7...
29594876    9563eda8d23992b79ef229562ea02f03085a9e2fb91642...
30646636    71e73b1fad436f5925e41529335b47226f524467426c95...
30893310    71e73b1fad436f5925e41529335b47226f524467426c95...
Name: customer_id, Length: 3257, dtype: object

In [28]:
for j,i in enumerate(vc.index.values[1000:1001]):
    print(j,i)
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    print(vc2)

0 534357002
706016001    264
534357008    247
400319005    229
448509014    225
640021005    197
            ... 
683639006      1
894593004      1
665082021      1
815456006      1
507883018      1
Name: article_id, Length: 42775, dtype: int64
